In [ ]:
from init_notebook import *
from src.train.experiment import load_experiment_trainer
from functools import partial

In [ ]:
from experiments.datasets import *


In [ ]:
def plot(ds, count=16*16):
    batch = next(iter(DataLoader(ds, batch_size=count)))
    if isinstance(batch, (tuple, list)):
        batch = batch[0]
    display(VF.to_pil_image(make_grid(batch, nrow=int(math.sqrt(count)))))

ds = cifar10_dataset(True)
plot(ds)

In [ ]:
def is_colorful(args):
    pic = args
    if isinstance(args, (tuple, list)):
        pic = args[0]
    pic = pic.flatten(1)
    s = pic.std(dim=1, keepdim=True)
    c = pic.mean(dim=1, keepdim=True)
    return s.mean().item() > .1 and c.std().item() > .05

dsf = ds.filter(is_colorful)
#print(len(list(dsf)))
plot(dsf, 32*32)


In [ ]:
ds2 = (
    all_image_patch_dataset(shape=(3, 64, 64))
    .filter(is_colorful)
    .shuffle(1000)
)

plot(ds2, 32*32)

In [ ]:
FILENAME = f"../datasets/colorful-uint-64x64.pt"

def store_dataset(
        images: Iterable,
        output_filename=FILENAME,
        max_megabyte=4_000,
):
    tensor_batch = []
    tensor_size = 0
    last_print_size = 0
    try:
        with tqdm(images) as progress:
            for image in images:
                if isinstance(image, (list, tuple)):
                    image = image[0]
                if len(image.shape) < 4:
                    image = image.unsqueeze(0)

                image = (image.clamp(0, 1) * 255).to(torch.uint8)
                tensor_batch.append(image)
                tensor_size += math.prod(image.shape) * 1

                progress.desc = f"{tensor_size:,}"
                progress.update()
                if tensor_size >= max_megabyte * 1024 * 1024:
                    break
    except KeyboardInterrupt:
        pass
    tensor_batch = torch.cat(tensor_batch)
    torch.save(tensor_batch, output_filename)

store_dataset(ds2)